## Section 1

### Importing all relevant packages

In [196]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import geocoder
import folium

### Defining methods

In [197]:
def get_long_lat(p_code):
    lat_lng_coords = None

    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(p_code))
      lat_lng_coords = g.latlng
        
    return lat_lng_coords

### Importing Wikipedia page as a BeautifulSoup object

In [198]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
content = response.text
soup = bs(content, "html.parser")

### Finding the table and all entries of that table

In [199]:
chart = soup.find("table", {"class": "wikitable sortable"})
entries = chart.find_all("tr")

### Creating list and filtering 'Not assigned" columns as well as joining duplicate Postal Codes

#### NOTE: Since the parsing of this data is index based: this is assuming that the columns stay in the same order on Wikipedia, if they change the order this code will not work properly

In [200]:
entries_list = []
for ent in entries[1:]:
    dup = False
    l = ent.text.strip().split("\n")
    if l[1] == "Not assigned" and l[2] == "Not assigned":
        continue
    elif l[2] == "Not assigned":
        l[2] = l[1]
    for entry in entries_list:
        if l[0] == entry[0]:
            entry[2] = ", ".join((entry[2], l[2]))
            dup = True
            break
    if not dup:    
        entries_list.append(ent.text.strip().split("\n"))

### Ceating DataFrame

In [201]:
postal_codes = pd.DataFrame(entries_list, columns=["PostalCode", "Borough", "Neighborhood"])

In [202]:
postal_codes.shape

(103, 3)

### Adding Latittude and Longitude to postal_codes DataFrame

#### NOTE: the commented out code tries to get long/lat data from geocoder method defined above. Geocoder was not working (timed out) so data was imported from .csv

In [203]:
# postal_codes.apply(lambda x: pd.Series(get_long_lat(x["PostalCode"]), index=['Latitude', 'Longitude']), axis=1)

In [204]:
long_lat_df = pd.read_csv(r"C:\Users\twmar\Downloads\Geospatial_Coordinates.csv")

In [205]:
postal_codes = postal_codes.set_index('PostalCode').join(long_lat_df.set_index('Postal Code'))

### Creating Map

In [218]:
canada_map = folium.Map(location=[43.70, -79.39], zoom_start=11)

for i, datum in postal_codes.iterrows():
    label = datum["Neighborhood"]
    lat, lng = datum["Latitude"], datum["Longitude"]
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(canada_map)  
    
canada_map